In [1]:
from trees import TreeMaker
from wrappers import GPTneoX_DenseWrapper, ActivationWrapper

import torch
import numpy as np

/Users/collinszczepanski/Code/cris/LLM/chenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Install 'desc2025.xml' from MeSH website
tm = TreeMaker('desc2025.xml')

In [3]:
# View MeSH online browser to select a base tree number
key = 'A'

G, names = tm.tree_from_key(key, max_depth=2)
print(names)

('Animal Structures', 'Body Regions', 'Cardiovascular System', 'Cells', 'Digestive System', 'Embryonic Structures', 'Endocrine System', 'Fluids and Secretions', 'Hemic and Immune Systems', 'Musculoskeletal System', 'Nervous System', 'Respiratory System', 'Sense Organs', 'Stomatognathic System', 'Tissues', 'Urogenital System', 'Plant Structures', 'Integumentary System', 'Viral Structures', 'Bacterial Structures', 'Fungal Structures', 'Air Sacs', 'Amniotic Fluid', 'Anal Sacs', 'Antibody-Producing Cells', 'Antigen-Presenting Cells', 'Beak', 'Biliary Tract', 'Blastomeres', 'Blood', 'Blood Cells', 'Blood Vessels', 'Blood-Brain Barrier', 'Blood-Retinal Barrier', 'Blood-Testis Barrier', 'Body Fluid Compartments', 'Body Fluids', 'Bone Marrow Cells', 'Branchial Region', 'Breast', 'Bursa of Fabricius', 'Cartilage', 'Cells, Cultured', 'Central Nervous System', 'Cheek', 'Chromaffin System', 'Chromosomes, Bacterial', 'Cleavage Stage, Ovum', 'Cloaca', 'Cloaca', 'Comb and Wattles', 'Connective Tissue

In [4]:
model = "EleutherAI/pythia-70m-deduped"
layer_num = 3

wrapper = ActivationWrapper(model)
layer = wrapper.make_layer_wrapper(layer_num, 'mlp')

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [5]:
wrapper.batch_logits(names, tokens = 'last').shape

torch.Size([38, 50278])

In [6]:
vocab_dict = wrapper.get_vocab()

In [7]:
print(vocab_dict)

{'Ġfatigue': 18693, 'Ġaffordable': 18261, 'Ġlowering': 23345, 'Ġrepresents': 6125, 'venile': 18044, 'Cass': 42231, 'ĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠ': 8369, 'eti': 45537, 'Ġfoc': 41685, 'Guy': 46963, 'Ġ****': 33684, '0038': 42861, 'OUS': 27832, 'atured': 30387, 'atisfied': 33496, 'Ġaccurately': 13613, 'ĠaprÃ¨s': 40797, 'Camera': 31962, 'kee': 19719, 'Ġconfidentiality': 39581, 'Ġfreedoms': 45971, 'Ġinterfaces': 19069, 'Ġpool': 6363, 'Ġtype': 1511, 'ĠAstr': 22042, 'ĠMedia': 11263, 'ĠVern': 30052, 'Ġsuggesting': 7738, 'ighting': 23144, 'Charlie': 45255, 'RTC': 46490, 'Fc': 43840, 'Ġsurjective': 44664, 'Ġ&=': 7196, "Ġ'<": 21083, 'ricia': 32055, 'Stat': 9710, 'vertisement': 27583, 'ĠBoeing': 35251, 'Ġhobby': 34057, 'Regular': 35616, 'Ġtracker': 40143, 'Ġtut': 17846, 'Ġvolunteer': 20848, 'Walker': 46949, 'UIC': 45717, '\\^[-': 47686, 'Ġliqu': 4874, 'Ġpersona': 35554, 'Ġsensations': 41868, 'Ġmah': 35926, 'Ġsatisfies': 12310, 'ajo

In [ ]:
all_acts, all_toks = layer.batch_activations(names)
#last_acts, last_toks = layer.batch_activations(names, tokens='last')

#print(layer.act)


#distances = torch.cdist(last_acts, last_acts, p=2)

In [ ]:
print(distances)